In [1]:
import geopandas as gpd
from shapely.geometry import LineString, Point
from shapely.ops import split, unary_union

# Identify riparian zones

Mathias has shapefiles defining (i) stream centre lines and (ii) sites of interest (one per stream). He needs to define riparian zones of approximately equal area, centred on the study sites.

For the moment, the strategy for defining buffers is as shown below.

<img src="../images/buffer_strategy.png" width="500" height="600">

On this diagram:

 * The blue line is the river and the red dot marks the study site.
 
 * The red circle is a buffer around the site of radius 25 m.
 
 * The blue dots are points on the river 25 m (in a straight line) up- and downstream from the study site.
 
 * The black lines are straight line segments between the study site and the up/downstream points.
 
 * Buffers are constructed around the straight line segments. The width of each buffer is 30 m plus half the mean stream width (measured during fieldwork) i.e. roughly 30 m from the bank of each stream.


In [2]:
# User options
buffer_width = 30  # m
segment_length = 25  # m
id_field = "Stream_id"
width_field = "s_width_m"

# File paths 
points_shp = r"../data/shapefiles/Stream_center_points.shp"
streams_shp = r"../data/shapefiles/Streams.shp"
buff_shp = r"../data/shapefiles/riparian_buffers.shp"

In [3]:
# Load the shapefiles
point_gdf = gpd.read_file(points_shp)
line_gdf = gpd.read_file(streams_shp)

# Initialize an empty list for the buffers
buffers_list = []

# Iterate over the streams
for idx, row in line_gdf.iterrows():
    stream_id = row[id_field]
    stream_width = row[width_field]
    line = row.geometry

    # Find the corresponding point
    point = point_gdf[point_gdf[id_field] == stream_id].geometry.iloc[0]

    # Buffer the point and find intersection with the line
    buffer = point.buffer(segment_length)
    intersection_line = line.intersection(buffer)
    if intersection_line.geom_type == "LineString":
        intersection_points = [
            Point(intersection_line.coords[0]),
            Point(intersection_line.coords[-1]),
        ]
    elif intersection_line.geom_type == "MultiLineString":
        intersection_points = [Point(line.coords[0]) for line in intersection_line]

    # Build the new line
    new_line = LineString([intersection_points[0], point, intersection_points[1]])

    # Buffer the line
    line_buffer = new_line.buffer(
        buffer_width + (stream_width / 2), join_style=3, cap_style=2
    )
    
    # Split the buffer into left and right sides of stream
    split_result = split(line_buffer, line)
    id_dict = {0:'a', 1:'b'}
    for idx, poly in enumerate(split_result.geoms):
        # Add the buffer to the list
        buffers_list.append(
            {id_field: f"{stream_id}{id_dict[idx]}", width_field: stream_width, "geometry": poly}
        )

# Convert the list to a GeoDataFrame
buffers = gpd.GeoDataFrame(
    buffers_list, columns=[id_field, width_field, "geometry"], crs=line_gdf.crs
)

# Save the buffers to a new shapefile
buffers.to_file(buff_shp)

buffers

,Stream_id,s_width_m,geometry
0,H20a,1.5,"POLYGON ((635939.273 6647308.998, 635938.505 6..."
1,H20b,1.5,"POLYGON ((635961.950 6647343.427, 635991.703 6..."
2,H21a,2.5,"POLYGON ((636732.508 6647383.455, 636736.375 6..."
3,H21b,2.5,"POLYGON ((636767.178 6647400.780, 636797.980 6..."
4,H22a,2.5,"POLYGON ((644322.676 6641289.760, 644324.383 6..."
5,H22b,2.5,"POLYGON ((644359.717 6641271.791, 644372.118 6..."
6,H23a,9.2,"POLYGON ((639117.876 6649631.583, 639112.926 6..."
7,H23b,9.2,"POLYGON ((639135.993 6649676.152, 639168.196 6..."
8,H24a,2.0,"POLYGON ((646811.725 6613147.674, 646811.632 6..."
9,H24b,2.0,"POLYGON ((646837.588 6613178.349, 646868.074 6..."


## Previous approach

The original approach was more complicated and involved tracing more precisely along the stream lines. This produces unwanted effects in some cases, so we've switched to the method above, but the code is preserved below in case it's useful in the future.

In [4]:
def split_line_with_point(line, point, tol=1e-6):
    """Split a LineString with a Point"""

    assert isinstance(line, LineString)
    assert isinstance(point, Point)

    # check if point is in the interior of the line
    if line.distance(point) > tol:
        # point not on line interior --> return collection with single identity line
        # (REASONING: Returning a list with the input line reference and creating a
        # GeometryCollection at the general split function prevents unnecessary copying
        # of linestrings in multipoint splitting function)
        return [line]
    elif line.coords[0] == point.coords[0]:
        # if line is a closed ring the previous test doesn't behave as desired
        return [line]

    # point is on line, get the distance from the first point on line
    distance_on_line = line.project(point)
    coords = list(line.coords)
    # split the line at the point and create two new lines
    # TODO: can optimize this by accumulating the computed point-to-point distances
    for i, p in enumerate(coords):
        pd = line.project(Point(p))
        if pd == distance_on_line:
            return [LineString(coords[: i + 1]), LineString(coords[i:])]
        elif distance_on_line < pd:
            # we must interpolate here because the line might use 3D points
            cp = line.interpolate(distance_on_line)
            ls1_coords = coords[:i]
            ls1_coords.append(cp.coords[0])
            ls2_coords = [cp.coords[0]]
            ls2_coords.extend(coords[i:])
            return [LineString(ls1_coords), LineString(ls2_coords)]


def trace_up_and_down(line_gdf, point_gdf, id_col, trace_dist, tol):
    """Trace up and down along a line for a specified distance from a point. The point must
    be within 'tol' of the line.
    """
    split_lines = []
    ids = []
    for idx, row in point_gdf.iterrows():
        # Find the line with the same 'Id'
        line = line_gdf[line_gdf[id_col] == row[id_col]].geometry.iloc[0]
        point = row.geometry

        # Find distance from start of line to point
        pos = line.project(point)

        # Create points up/downstream of this point
        point_before = line.interpolate(pos - trace_dist)
        point_after = line.interpolate(pos + trace_dist)

        # Split the line at the point_before
        split_line_before = split_line_with_point(line, point_before)

        # Iterate over the split line collection
        for segment in split_line_before:
            # Check if the segment contains the original point
            if segment.distance(point) < trace_dist / 100:
                # This is the segment to split again
                split_line_after = split_line_with_point(segment, point_after)

                # Iterate over the split line collection
                for final_segment in split_line_after:
                    # Check if the final_segment contains the original point
                    if final_segment.distance(point) < trace_dist / 100:
                        # This is the final segment to keep
                        split_lines.append(final_segment)
                        ids.append(row[id_col])

    split_lines = gpd.GeoDataFrame({id_col: ids}, geometry=split_lines)

    return split_lines

In [5]:
# User options
buffer_width = 30  # m
segment_length = 25  # m
tolerance = 1e3 # m
id_field = "Stream_id"
width_field = "s_width_m"
simplify = True

# File paths 
points_shp = r"../data/shapefiles/Stream_center_points.shp"
streams_shp = r"../data/shapefiles/Streams.shp"
buff_shp = r"../data/shapefiles/riparian_buffers_trace_streams.shp"

In [6]:
# Load the shapefiles
point_gdf = gpd.read_file(points_shp)
line_gdf = gpd.read_file(streams_shp)

reach_gdf = trace_up_and_down(line_gdf, point_gdf, id_field, segment_length, tolerance)
reach_gdf = reach_gdf.merge(
    line_gdf[[id_field, width_field]], how="left", on=id_field
)
if simplify:
    reach_gdf.geometry = reach_gdf.geometry.simplify(10, preserve_topology=True)
reach_gdf.crs = line_gdf.crs
riparian_gdf = reach_gdf.copy()
riparian_gdf.geometry = reach_gdf.geometry.buffer(
    buffer_width + reach_gdf[width_field] / 2, join_style=3, cap_style=2, mitre_limit=30
)

# Save
riparian_gdf.to_file(buff_shp)

riparian_gdf

,Stream_id,geometry,s_width_m
0,H37,"POLYGON ((646632.342 6562396.675, 646660.377 6...",6.0
1,H36,"POLYGON ((652545.270 6563003.690, 652596.354 6...",1.2
2,H35,"POLYGON ((644315.031 6643929.801, 644374.905 6...",1.5
3,H34,"POLYGON ((644842.630 6647551.953, 644886.182 6...",3.0
4,H33,"POLYGON ((643435.898 6648462.825, 643497.745 6...",3.3
5,H31,"POLYGON ((653176.070 6574830.753, 653151.706 6...",2.0
6,H30,"POLYGON ((654408.112 6567528.876, 654469.101 6...",1.0
7,H29,"POLYGON ((657053.353 6560180.184, 657085.861 6...",1.7
8,H28,"POLYGON ((644847.717 6607555.618, 644798.005 6...",3.0
9,H25,"POLYGON ((651241.081 6602958.333, 651298.914 6...",2.3
